In [3]:
!pip install -U torchaudio librosa jiwer datasets transformers huggingface_hub evaluate python-dotenv wandb speechbrain

  Using cached torchaudio-0.13.0-cp38-cp38-manylinux1_x86_64.whl (4.2 MB)
  Using cached librosa-0.9.2-py3-none-any.whl (214 kB)
  Using cached jiwer-2.5.1-py3-none-any.whl (15 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.7/451.7 kB 2.2 MB/s eta 0:00:00 MB/s eta 0:00:01:01
  Using cached transformers-4.24.0-py3-none-any.whl (5.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 kB 4.6 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.8 MB/s eta 0:00:00
  Using cached python_dotenv-0.21.0-py3-none-any.whl (18 kB)
  Using cached wandb-0.13.5-py2.py3-none-any.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.0/499.0 kB 3.2 MB/s eta 0:00:003.3 MB/s eta 0:00:01
  Using cached torch-1.13.0-cp38-cp38-manylinux1_x86_64.whl (890.2 MB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cache

In [6]:
import torch
import torchaudio
from speechbrain.pretrained import SpectralMaskEnhancement, SepformerSeparation

## Enhance audio

In [9]:
enhance_model = SpectralMaskEnhancement.from_hparams("speechbrain/metricgan-plus-voicebank")

In [11]:
?enhance_model.load_audio

In [13]:
# audio_path = "speechbrain/metricgan-plus-voicebank/example.wav"
audio_path = "/home/vpetukhov/other/Consulting/SEADS/FEM/survey_data/test_audio/test_hausa.wav"
noisy = enhance_model.load_audio(path=audio_path).unsqueeze(0)

# Add relative length tensor
enhanced = enhance_model.enhance_batch(noisy, lengths=torch.tensor([1.]))

# Saving enhanced signal on disk
torchaudio.save('enhanced.wav', enhanced.cpu(), 16000)

## Split speakers

In [14]:
model = SepformerSeparation.from_hparams(source="speechbrain/sepformer-wham", savedir='pretrained_models/sepformer-wham')

In [15]:
est_sources = model.separate_file(path=audio_path)

torchaudio.save("source1hat.wav", est_sources[:, :, 0].detach().cpu(), 8000)
torchaudio.save("source2hat.wav", est_sources[:, :, 1].detach().cpu(), 8000)

Resampling the audio from 16000 Hz to 8000 Hz
